In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=7114c3b7cda4c188c31a4ffedf38c66f0af6e81f7ca7c21f194f430f7c0ee41a
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 3.4MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.22671056193425942 ,random_state=0 , l1_ratio=0.98989898989899, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_7_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=896566, Mon Apr 13 21:46:04 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 9.944672832268057e+03 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 9.409727549938332e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.1
    3     51 9.083783708659577e+03 1.0e+00 2.29e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10230.03389666] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9409.72754994] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   85   1445 4.795247390816439e+01 1.3e+00 1.31e+00  1e+00  1e+00 0:03.1
  100   1700 -1.953154957341974e+03 1.3e+00 1.04e+00  1e+00  1e+00 0:03.7
  200   3400 -7.245128208602860e+03 1.4e+00 1.72e-01  2e-01  2e-01 0:07.4
  300   5100 -7.401819129032008e+03 1.4e+00 2.76e-02  3e-02  3e-02 0:10.9
  400   6800 -7.407184122948633e+03 1.4e+00 4.75e-03  5e-03  5e-03 0:14.5
  500   8500 -7.407285405126686e+03 1.4e+00 7.56e-04  7e-04  7e-04 0:18.1
  600  10200 -7.407289278964914e+03 1.4e+00 1.24e-04  1e-04  1e-04 0:21.7
  700  11900 -7.407289372696274e+03 1.4e+00 2.16e-05  2e-05  2e-05 0:25.3
  800  13600 -7.407289375810060e+03 1.4e+00 3.88e-06  4e-06  4e-06 0:28.9
  900  15300 -7.407289375877519e+03 1.4e+00 5.88e-07  5e-07  6e-07 0:32.5
 1000  17000 -7.407289375879396e+03 1.4e+00 1.04e-07  9e-08  1e-07 0:36.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -7.407289375879436e+03 1.4e+00 2.82e-08  3e-08  3e-08 0:39.6
 1200  20400 -7.407289375879447e+03 1.5e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9689.67013742] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9689.670137, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9689.670137, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

    2     34 8.721546321276495e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 7.378290734299205e+03 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1
   74   1258 -1.201211059336748e+03 1.3e+00 1.26e+00  1e+00  1e+00 0:03.1
  100   1700 -2.530121682853687e+03 1.3e+00 1.02e+00  1e+00  1e+00 0:04.2
  200   3400 -5.807173855664388e+03 1.3e+00 3.40e-01  3e-01  3e-01 0:08.5
  300   5100 -6.594451784228332e+03 1.4e+00 1.14e-01  1e-01  1e-01 0:12.7
  400   6800 -6.735693732830515e+03 1.8e+00 6.99e-02  6e-02  8e-02 0:16.9
  500   8500 -6.823729495979599e+03 2.4e+00 6.18e-02  5e-02  8e-02 0:21.2
  600  10200 -6.875695868228252e+03 2.9e+00 4.73e-02  4e-02  6e-02 0:25.4
  700  11900 -6.892344685056693e+03 3.1e+00 1.95e-02  2e-02  3e-02 0:29.7
  800  13600 -6.896278513679014e+03 3.3e+00 1.05e-02  9e-03  1e-02 0:34.0
  900  15300 -6.897889750056946e+03 3.6e+00 6.02e-03  5e-03  8e-03 0:38.2
 1000  17000 -6.898321990475691e+03 3.9e+00 3.55e-03  3e-03  5e-03 0:42.5
Iterat #Fevals   function value  axis ra

...

   84   1428 -1.072380937428779e+03 1.3e+00 1.11e+00  1e+00  1e+00 0:03.1
  100   1700 -2.987686850607965e+03 1.3e+00 9.15e-01  9e-01  9e-01 0:03.7
  200   3400 -7.266676002453338e+03 1.3e+00 1.37e-01  1e-01  1e-01 0:07.3
  300   5100 -7.402691226003530e+03 1.3e+00 2.43e-02  2e-02  2e-02 0:11.0
  400   6800 -7.407166936233671e+03 1.3e+00 4.39e-03  4e-03  4e-03 0:14.6
  500   8500 -7.407286888601278e+03 1.3e+00 6.41e-04  6e-04  6e-04 0:18.2
  600  10200 -7.407289276350277e+03 1.4e+00 1.23e-04  1e-04  1e-04 0:21.9
  700  11900 -7.407289373040154e+03 1.4e+00 2.36e-05  2e-05  2e-05 0:25.5
  800  13600 -7.407289375795937e+03 1.4e+00 3.59e-06  3e-06  3e-06 0:29.2
  900  15300 -7.407289375876846e+03 1.4e+00 6.21e-07  6e-07  6e-07 0:32.8
 1000  17000 -7.407289375879374e+03 1.4e+00 1.06e-07  1e-07  1e-07 0:36.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -7.407289375879433e+03 1.4e+00 3.87e-08  3e-08  4e-08 0:40.0
 1200  20400 -7.407289375879440e+03 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9689.67013742] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9689.670137, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9689.670137, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

    2     34 7.681522093888052e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 6.601776032548944e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10024.46391538] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   71   1207 -1.148394160315810e+03 1.3e+00 1.29e+00  1e+00  1e+00 0:03.1
  100   1700 -1.991306510777290e+03 1.3e+00 9.97e-01  1e+00  1e+00 0:04.4
  200   3400 -5.238467418231424e+03 1.3e+00 3.33e-01  3e-01  3e-01 0:08.8
  300   5100 -6.271160344764768e+03 1.5e+00 1.55e-01  1e-01  2e-01 0:13.2
  400   6800 -6.532879666081376e+03 1.8e+00 8.69e-02  8e-02  1e-01 0:17.6
  500   8500 -6.597577319268282e+03 2.1e+00 4.14e-02  3e-02  5e-02 0:22.1
  600  10200 -6.616088250408856e+03 2.4e+00 2.35e-02  2e-02  3e-02 0:26.5
  700  11900 -6.621155538832551e+03 2.7e+00 1.40e-02  1e-02  2e-02 0:30.9
  800  13600 -6.623039065903345e+03 3.2e+00 6.99e-03  5e-03  1e-02 0:35.3
  900  15300 -6.623667118657423e+03 3.9e+00 4.79e-03  4e-03  9e-03 0:39.7
 1000  17000 -6.623838574777757e+03 4.2e+00 2.02e-03  1e-03  4e-03 0:44.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -6.623885829711903e+03 4.4e+00 1.38e-03  1e-03  3e-03 0:48.4
 1200  20400 -6.623897915689696e+03 4.5e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F7_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:100])
    SVM_Fun [i] = F7(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F7(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2083.36456061]), count=array([30])) 0.0
SVM
ModeResult(mode=array([6581.33977416]), count=array([1])) 1742.818373951988
ELN
ModeResult(mode=array([7112.06044821]), count=array([3])) 1220.8204508115641
